In [38]:
import pandas as pd
import os
from sqlalchemy import text
from sqlalchemy.engine import URL

from google.cloud import bigquery

In [39]:
# Initialize the BigQuery client using ADC
client = bigquery.Client()

# Test the connection by listing your BigQuery datasets
datasets = list(client.list_datasets())
project = client.project

if datasets:
    print(f"Datasets in project {project}:")
    for dataset in datasets:
        print(f"\t{dataset.dataset_id}")
else:
    print(f"No datasets found in project {project}.")


Datasets in project physionet-data-435019:
	mimiciii


## Patient Query

In [44]:
patientQuery = """SELECT DISTINCT
    p.SUBJECT_ID,
    p.GENDER,
    p.DOB,
    p.DOD,
    p.EXPIRE_FLAG,
    a.HADM_ID,
    a.ADMITTIME,
    a.DISCHTIME,
    a.ADMISSION_TYPE,
    DATE_DIFF(DATE(a.ADMITTIME), DATE(p.DOB), YEAR) AS AGE_AT_ADMISSION
FROM
    mimiciii.patients p
JOIN
    mimiciii.diagnoses_icd d
    ON p.SUBJECT_ID = d.SUBJECT_ID
JOIN
    mimiciii.admissions a
    ON d.HADM_ID = a.HADM_ID
WHERE
    d.ICD9_CODE IN ('99591', '99592');
"""

patient_df = client.query(patientQuery).to_dataframe()

## Heart Rate Query

In [45]:
heart_rate_query = """
SELECT C.SUBJECT_ID, AVG(C.VALUENUM) AS HEARTRATE_MEAN
FROM mimiciii.chartevents AS C
INNER JOIN mimiciii.diagnoses_icd AS D ON D.SUBJECT_ID = C.SUBJECT_ID
JOIN mimiciii.d_items AS ITEM ON ITEM.ITEMID = C.ITEMID
WHERE D.ICD9_CODE IN ('99591', '99592') AND (C.ITEMID  IN (211, 220045))
GROUP BY C.SUBJECT_ID, ITEM.LABEL
ORDER BY C.SUBJECT_ID ASC
"""

heart_rate_df = client.query(heart_rate_query).to_dataframe()

In [43]:
merge = pd.merge(patient_df, heart_rate_df, how='left', on='SUBJECT_ID')

merge.head()

,SUBJECT_ID,GENDER,DOB,DOD,EXPIRE_FLAG,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,AGE_AT_ADMISSION,HEARTRATE_MEAN
0,18333,F,2094-01-21,2168-01-08,1,185644,2167-12-23 20:54:00,2168-01-08 02:40:00,EMERGENCY,73,76.692308
1,55935,F,2053-10-02,2112-02-08,1,116770,2112-01-30 23:04:00,2112-02-08 05:20:00,EMERGENCY,59,100.165899
2,32012,F,2058-04-26,2144-02-08,1,138125,2144-02-01 05:13:00,2144-02-08 15:05:00,EMERGENCY,86,94.541401
3,8060,F,2111-11-26,2164-02-08,1,125303,2164-01-04 21:26:00,2164-02-08 12:00:00,EMERGENCY,53,103.072883
4,17634,F,2057-09-03,2116-02-16,1,169434,2116-02-14 17:30:00,2116-02-16 09:40:00,EMERGENCY,59,110.307692
